IGENIERÍA DE DATOS

INGESTA DE DATOS

In [1]:
#Importo la librería pandas y gzip (cabe aclarar que se debe tener instalado pandas previamente, para eso utilizo el comando pip install pandas)
import pandas as pd
import gzip
import numpy as np
import pyarrow as pa

Decido utilizar los 3 archivos directamente en formato comprimido, para el caso del archivo "steam_games", se puede leer directamente utilizando
gzip pero para los siguientes 2 archivos: "user_reviews" y "user_items" hubo que hacer una función ya que el json venía con comillas simples

Ingesta de datos del primer archivo "steam_games"

In [2]:
#con gzip abrimos el json y generamos un dataframe
with gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/steam_games.json.gz', 'rt', encoding='utf-8') as steam_games:
       
    df_steam_games = pd.read_json(steam_games, lines=True)

In [3]:
steam_games = df_steam_games.drop(columns = ['items','user_id','steam_id','items_count','tags'])

In [4]:
steam_games.dropna(inplace = True, how='all')

In [5]:
#realizo una función para separar la columna "release_date" en year, month y day para luego consumir en las funciones
def procesar_fecha(fecha):
    if fecha and fecha != 'none':
        partes = fecha.split("-")
        if len(partes) == 3:
            año = partes[0]
            mes = partes[1]
            dia = partes[2]
            return año, mes, dia
    return None, None, None

# Aplicp la función a la columna "release_date" y creo columnas separadas para año, mes y día
steam_games[['release_year', 'release_month', 'release_day']] = steam_games['release_date'].apply(lambda x: pd.Series(procesar_fecha(x)))

# Elimino la columna original "release_date"
steam_games.drop(columns=['release_date'], inplace=True)


Ingesta de datos del archivo "user_reviews" obteniendo un df_reviews_desanidado

In [6]:
#Ahora genero un dataframe para "user_reviews" utilizando una función que recorre y completa una lista vacía con la cual genero el DF
import ast #importo la librería ast a utilizar

info = [] 

for i in gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/user_reviews.json.gz'):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_user_reviews = pd.DataFrame(info) 


In [7]:
# Con las funciones explode y normalize desanidamos las columna "reviews" para poder
# disponibilizar esos datos para consumirlos en etapas posteriores
df_reviews_exploded = df_user_reviews.explode('reviews')
df_reviews_desanidado = pd.json_normalize(df_reviews_exploded['reviews'].dropna())

# Por otra parte reindexamos los DF
df_reviews_desanidado.reset_index(inplace=True)
df_reviews_exploded.reset_index(inplace=True)

# Concatenamos y eliminamosla columna que originalmente desanidamos

user_reviews = pd.concat([df_reviews_exploded,df_reviews_desanidado], axis=1)
user_reviews = user_reviews.drop(columns = ['reviews'])

In [8]:
#verifico y borro los duplicados
user_reviews_duplicados =  user_reviews.duplicated( keep="first")
user_reviews_duplicados.sum()

10

In [9]:
user_reviews = user_reviews.drop_duplicates(keep = "first")

In [10]:
#realizo una función para separar la columna "posted" en dia, mes y año para luego consumir en las funciones

def procesar_fecha(data):
    import re
    if isinstance(data, str):
        match = re.search(r'(\w+) (\d+), (\d+)', data)
        if match:
            mes = match.group(1)
            dia = match.group(2)
            año = match.group(3)
            meses = {
                'January': '01',
                'February': '02',
                'March': '03',
                'April': '04',
                'May': '05',
                'June': '06',
                'July': '07',
                'August': '08',
                'September': '09',
                'October': '10',
                'November': '11',
                'December': '12'
            }
            mes_numero = meses.get(mes)
            if mes_numero is not None:
                return mes_numero, dia, año
    return None, None, None

# Aplico la función a la columna "posted" después de convertirla en cadenas de texto y unifico la fecha en el formato
#YYYY-MM-DD para consumir en las funciones
user_reviews[['Mes', 'Día', 'Año']] = user_reviews['posted'].astype(str).apply(lambda x: pd.Series(procesar_fecha(x)))
user_reviews['Año'].fillna(0, inplace=True)
user_reviews['Mes'].fillna(0, inplace=True)
user_reviews['Día'].fillna(0, inplace=True)


# Convierte las columnas 'Año', 'Mes' y 'Día' en enteros
user_reviews['Año'] = user_reviews['Año'].astype(int)
user_reviews['Año'] = user_reviews['Año'].fillna(0).astype(int)
user_reviews['Mes'] = user_reviews['Mes'].astype(int)
user_reviews['Mes'] = user_reviews['Mes'].fillna(0).astype(int)
user_reviews['Día'] = user_reviews['Día'].astype(int)
user_reviews['Día'] = user_reviews['Día'].fillna(0).astype(int)
# Define una fecha predeterminada para las filas con valores cero
fecha_predeterminada = pd.to_datetime('1900-01-01')

# Crea una nueva columna 'fecha_review' con la fecha predeterminada para las filas con valores cero
user_reviews['fecha_review'] = user_reviews.apply(lambda row: pd.to_datetime(f"{row['Año']}-{row['Mes']}-{row['Día']}", format='%Y-%m-%d', errors='coerce') if row['Año'] != 0 and row['Mes'] != 0 and row['Día'] != 0 else fecha_predeterminada, axis=1)

# Elimino la columna original "posted"
user_reviews.drop(columns=['posted'], inplace=True)


Ahora voy a realizar el análisis de sentimiento con NLP utilizando la librería NLTK

In [11]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Al ejecutar estos comandos, me encontré con errores por los certificados SSL, y tuve que instalar de otra forma como
muestro en el siguiente Markdown
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [12]:
#defino la función para analizar los sentimientos
def analizar_sentimiento(texto):
    if isinstance(texto, str):
        sia = SentimentIntensityAnalyzer()
        sentimiento = sia.polarity_scores(texto)
    
        # Determinar la etiqueta en función de la puntuación compuesta
        if sentimiento['compound'] >= 0.05:
            return '2'
        elif sentimiento['compound'] <= -0.05:
            return '0'
        else:
            return '1'
    else:
            return '1'

In [13]:
user_reviews['sentimiento'] = user_reviews['review'].apply(analizar_sentimiento)

In [14]:
#Borro registros con valores nulos en la columna "review" y duplicados
user_reviews = user_reviews.dropna(subset=['review'])
user_reviews = user_reviews.drop_duplicates( keep = "first")

In [15]:
#Elimino la columma "index"
user_reviews = user_reviews.drop(columns = ['index'])

Ingesta de datos del archivo "user_items" obteniendo un df_items_desanidado

In [16]:
#Ahora genero un dataframe para "users_items" utilizando una función que recorre y completa una lista vacía con la cual genero el DF
import ast #importo la librería ast a utilizar

info = [] 

for i in gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/users_items.json.gz'):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_users_items = pd.DataFrame(info) 

In [17]:
# Con las funciones explode y normalize desanidamos las columna "items" para poder
# disponibilizar esos datos para consumirlos en etapas posteriores
df_items_exploded = df_users_items.explode('items')
df_items_desanidado = pd.json_normalize(df_items_exploded['items'])

# Por otra parte reindexamos los DF
df_items_exploded.reset_index(inplace=True)
df_items_desanidado.reset_index(inplace=True)

# Concatenamos y eliminamos la columna que previamente desanidamos
user_items = pd.concat([df_items_exploded,df_items_desanidado], axis=1)
user_items = user_items.drop(columns = ['items'])

In [18]:
#identifico valores duplicados, como no hay registros duplicados, no es necesario borrar
user_items_duplicados =  user_items.duplicated( keep="first")
user_items_duplicados.sum()

0

In [19]:
#borro los duplicados
user_items = user_items.drop_duplicates(keep = "first")

Uno los datasets (DataFrames) user_items & steam_games ya que su info es complementaria y necesaria para las funciones
que debemos armar pero primero cambio el tipo de dato a float para poder unirlos

In [20]:
user_items['item_id']=user_items['item_id'].astype(float)

In [26]:
steam_games['item_id']=user_items['item_id'].astype(float)

NameError: name 'user_items' is not defined

In [22]:
user_items_steam_games = user_items.merge(steam_games, left_on='item_id',right_on='id',how='left')

Para la columna precio, defino que si el valor no es numérico, sea 0 para evitar errores de cálculos

In [27]:
user_items_steam_games['price'] = user_items_steam_games['price'].apply(lambda x: 0 if isinstance(x, str) else (x if pd.notna(x) else np.nan))


NameError: name 'user_items_steam_games' is not defined

TRATAMIENTO DE DATOS

In [24]:
#borro las columnas "index" del dataframe user_items_steam_games
user_items_steam_games = user_items_steam_games.drop(columns = ['index'])

In [25]:
#Al intentar exportar el df a parquet, encontré que en la columna "metascore" tenía valores "none" que
#me generaban error ya que no es un Nan, por lo que decido reemplazarlos por NA

# Reemplazo 'NA' con NaN en la columna 'metascore'
user_items_steam_games['metascore'] = user_items_steam_games['metascore'].replace('NA', np.nan)

In [26]:
#Decido borrar las columnas playtime_2weeks, specs y publisher para reducir espacio en el dataframe user_items_steam_games
user_items_steam_games = user_items_steam_games.drop(columns = ['playtime_2weeks'])
user_items_steam_games = user_items_steam_games.drop(columns = ['specs'])
user_items_steam_games = user_items_steam_games.drop(columns = ['publisher'])

Luego de confeccionados los dataframes con la info que necesito para consumir en las funciones,
decido guardarlos en archivos parquet y no en csv para optimizar 

Para eso primero instalo la biblioteca pyarrow con el comando pip install pyarrow

In [28]:
#renombro los dataframes
reviews= user_reviews
users_games = user_items_steam_games


Para la API tuve que instalar fastapi y uvicorn para poder crear el servidor web (consultar documentación en https://fastapi.tiangolo.com)

Feature engineering

Uno de los pedidos para este proyecto fue aplicar un análisis de sentimiento a los reviews de los usuarios. Para ello se creó una nueva columna llamada 'sentiment_analysis' que reemplaza a la columna que contiene los reviews donde clasifica los sentimientos de los comentarios con la siguiente escala:

0 si es malo,
1 si es neutral o esta sin review
2 si es positivo.
Dado que el objetivo de este proyecto es realizar una prueba de concepto, se realiza un análisis de sentimiento básico utilizando TextBlob que es una biblioteca de procesamiento de lenguaje natural (NLP) en Python. El objetivo de esta metodología es asignar un valor numérico a un texto, en este caso a los comentarios que los usuarios dejaron para un juego determinado, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo.

Esta metodología toma una revisión de texto como entrada, utiliza TextBlob para calcular la polaridad de sentimiento y luego clasifica la revisión como negativa, neutral o positiva en función de la polaridad calculada. En este caso, se consideraron las polaridades por defecto del modelo, el cuál utiliza umbrales -0.2 y 0.2, siendo polaridades negativas por debajo de -0.2, positivas por encima de 0.2 y neutrales entre medio de ambos.

Por otra parte, y bajo el mismo criterio de optimizar los tiempos de respuesta de las consultas en la API y teniendo en cuenta las limitaciones de almacenamiento en el servicio de nube para deployar la API, se realizaron dataframes auxiliares para cada una de las funciones solicitadas. En el mismo sentido, se guardaron estos dataframes en formato parquet que permite una compresión y codificación eficiente de los datos.

Todos los detalles del desarrollo se pueden ver en la Jupyter Notebook 01d_Feature_eng.

Levanto los archivos parquet exportados luego de la limpieza de datos realizada en la etapa de Data Engineer (consultar archivo Data Engineer.ipynb)

In [29]:
#genero un archivo parquet con el dataframe "reviews" para utilizar en las funcione de la api
reviews.to_parquet('user_reviews.parquet', engine='pyarrow', compression='snappy')

In [30]:
#genero un dataframe para consumir en la función userdata
df_userdata = users_games[['user_id','price']]
df_userdata.to_parquet('df_userdata.parquet', engine='pyarrow', compression='snappy')

In [31]:
#genero un dataframe para consumir en la función genre_ranking
users_games_genero = users_games[['genres','playtime_forever']]
 #Realizo un explode de la columna 'genres' para poder desanidar la información en distintos registros
users_games_generos = users_games_genero[['genres','playtime_forever']].explode('genres')
    
# Agrupo por género y sumo los valores de "playtime_forever"
agrupado = users_games_generos.groupby('genres')['playtime_forever'].sum().reset_index()

# Ordeno por la suma de "playtime_forever" en orden descendente
agrupado = agrupado.sort_values(by='playtime_forever', ascending=False)
agrupado.to_parquet('generos.parquet', engine='pyarrow', compression='snappy')


In [42]:
#genero un nuevo dataframe con las columnas genre, playtime_forever, user_id y user_url. Lo exporto a parquet para luego consumirlo
df_generos_horas = users_games[['user_id','user_url','genres','playtime_forever']]
#Realizo un explode de la columna 'genres' para poder desanidar la información en distintos registros
df_generos_horas = df_generos_horas.explode('genres')
    
# Agrupo por usuario y sumo las horas de juego para cada usuario
df_generos_horas = df_generos_horas.groupby(['genres','user_id', 'user_url'])['playtime_forever'].sum().reset_index()

df_generos_horas.to_parquet('generos_horas.parquet', engine='pyarrow', compression='snappy')

In [43]:
df_generos_horas

,genres,user_id,user_url,playtime_forever
0,Action,--000--,http://steamcommunity.com/id/--000--,139469.0
1,Action,--ace--,http://steamcommunity.com/id/--ace--,69325.0
2,Action,--ionex--,http://steamcommunity.com/id/--ionex--,38315.0
3,Action,-2SV-vuLB-Kg,http://steamcommunity.com/id/-2SV-vuLB-Kg,42500.0
4,Action,-404PageNotFound-,http://steamcommunity.com/id/-404PageNotFound-,117423.0
...,...,...,...,...
704777,Web Publishing,zepavil,http://steamcommunity.com/id/zepavil,37926.0
704778,Web Publishing,zeshirky,http://steamcommunity.com/id/zeshirky,1.0
704779,Web Publishing,zevlupine,http://steamcommunity.com/id/zevlupine,4.0
704780,Web Publishing,zilaman,http://steamcommunity.com/id/zilaman,9.0


In [37]:
df_generos_horas

,user_id,user_url,playtime_forever
0,--000--,http://steamcommunity.com/id/--000--,229119.0
1,--ace--,http://steamcommunity.com/id/--ace--,91837.0
2,--ionex--,http://steamcommunity.com/id/--ionex--,56735.0
3,-2SV-vuLB-Kg,http://steamcommunity.com/id/-2SV-vuLB-Kg,83813.0
4,-404PageNotFound-,http://steamcommunity.com/id/-404PageNotFound-,477522.0
...,...,...,...
87621,zzonci,http://steamcommunity.com/id/zzonci,1716.0
87622,zzoptimuszz,http://steamcommunity.com/id/zzoptimuszz,432433.0
87623,zzydrax,http://steamcommunity.com/id/zzydrax,9361.0
87624,zzyfo,http://steamcommunity.com/id/zzyfo,86455.0


In [50]:
#genero un dataframe para consumir en la función developer
df_developer= users_games[['developer','price','release_year','item_id_x']]
df_developer.to_parquet('df_developer.parquet', engine='pyarrow', compression='snappy')

 # Filtro el DataFrame original por desarrollador y seleccionar columnas relevantes
developer_df = users_games[['developer', 'price', 'release_year', 'item_id_x']]

# Calculo la cantidad de ítems gratuitos (Free) por año para todos los desarrolladores
developer_df['items_free_por_anio'] = developer_df['price'].apply(lambda x: 1 if x == 0 else 0)
items_free_por_anio = developer_df.groupby(['developer', 'release_year'])['items_free_por_anio'].sum().reset_index()

# Calculo la cantidad total de ítems por año para todos los desarrolladores
items_totales_por_anio = developer_df.groupby(['developer', 'release_year'])['item_id_x'].nunique().reset_index()

# Exporto el DataFrame a un archivo Parquet
df_developer = items_free_por_anio.merge(items_totales_por_anio, on=['developer', 'release_year'])
df_developer.to_parquet('df_developer.parquet', engine='pyarrow', compression='snappy')

/var/folders/_x/y99l7l9s60q1726y6701xbkw0000gn/T/ipykernel_81137/3496855238.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  developer_df['items_free_por_anio'] = developer_df['price'].apply(lambda x: 1 if x == 0 else 0)


In [6]:
#ajusto la columna price, defino que si el valor no es numérico, sea 0 para evitar errores de cálculos
steam_games['price'] = steam_games['price'].apply(lambda x: 0 if isinstance(x, str) else (x if pd.notna(x) else np.nan))

In [7]:
steam_games

,publisher,genres,app_name,title,url,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,release_year,release_month,release_day
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,None,Kotoshiro,2018,01,04
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,None,Secret Level SRL,2018,01,04
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,None,Poolians.com,2017,07,24
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...,0.83,[Single-player],0.99,0.0,767400.0,None,彼岸领域,2017,12,07
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,http://steamcommunity.com/app/773570/reviews/?...,1.79,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,None,"Nikita ""Ghost_RUS""",2018,01,04
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,None,Sacada,2018,01,04
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich,2018,01,04
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",2017,09,02


In [29]:
steam_games = steam_games[['genres','price','early_access','id','release_year','publisher','app_name','title','developer']]

In [30]:
steam_games

,genres,price,early_access,id,release_year,publisher,app_name,title,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",4.99,0.0,761140.0,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",0.00,0.0,643980.0,2018,"Making Fun, Inc.",Ironbound,Ironbound,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",0.00,0.0,670290.0,2017,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com
88313,"[Action, Adventure, Casual]",0.99,0.0,767400.0,2017,彼岸领域,弹炸人2222,弹炸人2222,彼岸领域
88314,None,2.99,0.0,773570.0,None,None,Log Challenge,None,None
...,...,...,...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",1.99,0.0,773640.0,2018,Ghost_RUS Games,Colony On Mars,Colony On Mars,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",4.99,0.0,733530.0,2018,Sacada,LOGistICAL: South Africa,LOGistICAL: South Africa,Sacada
120442,"[Indie, Racing, Simulation]",1.99,0.0,610660.0,2018,Laush Studio,Russian Roads,Russian Roads,Laush Dmitriy Sergeevich
120443,"[Casual, Indie]",4.99,0.0,658870.0,2017,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,"xropi,stev3ns"


In [20]:
#exporto el DF steam_games para realizar el EDA luego
steam_games.to_parquet('steam_games.parquet', engine='pyarrow', compression='snappy')